## Importing libraries

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from collections import Counter
import csv

## Functions

In [2]:
def fuzz1(list1, n):
    
    ## Calculate the clones for BCRs
    
    grs = list() # groups of names with distance > n
    for name in list1:
        for g in grs:
            if all(fuzz.ratio(name, w) > n for w in g):
                g.append(name)
                break
        else:
            grs.append([name, ])
    return grs


def group2df_list (grouped):
    
    ## Combine the information of the clonal clusters to the dataframe
    
    nns = pd.DataFrame(grouped)
    nns.reset_index(inplace = True)
    df = pd.DataFrame(nns)
    df =df.rename(columns={"index": "Cluster"})
    data = []
    for i in df.itertuples():
        lst = i[2]
        for col2 in lst:
            data.append([i[1], col2])

    ## List into a dataframe
    df_output = pd.DataFrame(data =data, columns=df.columns)
    df_output.reset_index(inplace = True)
    return df_output

## Loading data

In [15]:
### Data should include the amino-acid and nucleotide sequences of the CDR3 region
dataset = pd.read_csv("BCR_CONTIG_productive", sep='\t', lineterminator='\n', names=None)

In [16]:
dataset

,Unnamed: 0,SequenceID,VDOMAIN_Functionality,VGENE_allele,VREGION_score,VREGION_identity,VREGION_identity_nt,VREGION_INS_DEL_identity,VREGION_INS_DEL_identity_nt,JGENE_allele,...,FR4_IMGT,FR1_IMGT_AA,CDR1_IMGT_AA,FR2_IMGT_AA,CDR2_IMGT_AA,FR3_IMGT_AA,CDR3_IMGT_AA,JUNCTION_AA,J_REGION_AA,FR4_IMGT_AA
0,0,AAAGTAGAGACTAGAT-1_contig_1,productive,"Homsap IGKV1-39*01 F, or Homsap IGKV1D-39*01 F",1269.0,94.98,265/279 nt,NaN,NaN,Homsap IGKJ3*01 F,...,ttcggccctgggaccaaagtggatatcaagc,DIQMTQSPSSLSASVGDRVTITCRAS,QSI......STY,LNWYQQKPGKAPKFLIS,AA.......S,SLQNGVP.SRFSGSG..SGTEFTLTISSVQPEDFASYYC,QQSHSTPFT,CQQSHSTPFTF,FTFGPGTKVDIK,FGPGTKVDIK
1,1,AAAGTAGAGACTAGAT-1_contig_2,productive (see comment),Homsap IGHV4-61*01 F,1275.0,93.13,271/291 nt,92.78,270/291 nt,Homsap IGHJ6*02 F,...,tggggccaagggaccacgcttatagtctcctca,QVQLQESGP.GLVKPSETLSLTCTVS,GASVS..SGTYF,WSWVRQPPGKGLEWIGY,IYPS...GST,NYNPSRK.SPVSISIDTSKNQFSLKLSSVTAADTAVYYC,ARGAYERRVTMGYYGMDV,CARGAYERRVTMGYYGMDVW,YYGMDVWGQGTTLIVSS,WGQGTTLIVSS
2,3,AACCATGCAAGACGTG-1_contig_1,productive,Homsap IGHV4-61*01 F,1239.0,91.75,267/291 nt,NaN,NaN,Homsap IGHJ6*02 F,...,tggggcctagggaccacggtcaccgtctccgca,QVQLQESGP.GLVRPSETLTLTCTVS,GGSVS..SGTYF,WSWIRQPPGKGLEWVGY,IYYS...GTT,SYNPSVK.SRVAIASDRSKNQVSLRLTSVTAADTAMYYC,ARGVIRYSHFFYGLDV,CARGVIRYSHFFYGLDVW,FYGLDVWGLGTTVTVSA,WGLGTTVTVSA
3,4,AACCATGCAAGACGTG-1_contig_2,productive,Homsap IGKV3-20*01 F,1257.0,93.97,265/282 nt,NaN,NaN,Homsap IGKJ4*01 F,...,ttcggcggagggaccaaggtggagatcaaac,GTVLTQSPGSLSLSPGESVTLSCRAS,QGIS.....SSY,VAWYQHKPGQAPRLLIY,GA.......S,SRATGIP.DRFSGSG..SGTDFTLTISSLEPEDFAVYYC,QQFGSSPPLT,CQQFGSSPPLTF,LTFGGGTKVEIK,FGGGTKVEIK
4,6,ACAGCTAAGGGATCTG-1_contig_1,productive,Homsap IGHV1-2*02 F,1188.0,90.28,260/288 nt,NaN,NaN,Homsap IGHJ4*02 F,...,tggggccagggaaccctggtctccgtctcctcag,QVHLVQSGA.EIKKPGASVKVSCKAS,GYIF....IDSY,IHWVRQAPGQGLEWMGW,INPK..SGGT,NFAEKFQ.GRVTLTRDTSISTAYMEVRRLTSDDTAVYYC,AREVRAGNRIVFWLDY,CAREVRAGNRIVFWLDYW,DYWGQGTLVSVSS,WGQGTLVSVSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13752,16692,TCATTACAGGTAAACT-1_contig_1,productive,Homsap IGLV3-1*01 F,1341.0,97.85,273/279 nt,NaN,NaN,Homsap IGLJ1*01 F,...,ttcggaactgggacccgggtcaccgtcctag,SYDLTQPPS.VSVSPGQTASITCSGD,KLG......DKY,ACWYQQKPGQSPVLVLY,QD.......T,KRPSGIP.ERFSGSN..SGKTATLTISGTQAMDEADYYC,QAWDSNLRV,CQAWDSNLRVF,VFGTGTRVTVL,FGTGTRVTVL
13753,16694,TGCGGGTAGGACACCA-1_contig_1,productive,Homsap IGKV1-9*01 F,1278.0,95.34,266/279 nt,NaN,NaN,Homsap IGKJ3*01 F,...,ttcggccctgggaccagagtggatttcaaac,DIQLTQSPSFLSASVGDRVTITCRAS,QGI......SSH,LAWYQQKPGRAPKLLIS,AA.......S,TLQSWVP.FRFSGRR..SGTEFTLTISSLQPEDFATYYC,QQLNSYPFT,CQQLNSYPFTF,FTFGPGTRVDFK,FGPGTRVDFK
13754,16695,TGCGGGTAGGACACCA-1_contig_2,productive,Homsap IGHV2-5*01 F,1140.0,87.97,256/291 nt,NaN,NaN,Homsap IGHJ4*02 F,...,tggggccagggaatcctggtcatcgtctcctccg,QITLKESGP.TLVKPTQTLTLTCTYS,GFTLG..STGVG,VGWIRQPPGKPLEWLAL,IHWN...EDK,RYTPSLE.TRLTITKDMSETHVVLTMTRMEPGDTATYYC,ARRVGTTAPFDY,CARRVGTTAPFDYW,FDYWGQGILVIVSS,WGQGILVIVSS
13755,16698,TGGCTGGAGTTGCAGG-1_contig_1,productive,"Homsap IGHV3-23*01 F, or Homsap IGHV3-23D*01 F",1305.0,94.79,273/288 nt,NaN,NaN,Homsap IGHJ4*02 F,...,tggggccagggaaccctggtcaccgtctcctcag,EVQLLESGG.GLVQSGGSLRLSCAAS,GFTF....SRYD,MSWVRQAPGKGLECVSV,ISGN..GDRT,YYADSVK.GRFTISRDNSKNTLYLQMNSLRAEDTAVYYC,AKDAIDADSSGYYSDY,CAKDAIDADSSGYYSDYW,YSDYWGQGTLVTVSS,WGQGTLVTVSS


### Fixing the VGENE_Gene column and creating a column with only V Family Information

In [17]:
dataset['VGENE_allele1']= dataset['VGENE_allele'].str.split(' ').str[1]
dataset['VGENE_Gene']= dataset['VGENE_allele1'].str.split('*').str[0]
dataset['VFAM']= dataset['VGENE_Gene'].str.split('-').str[0]

In [18]:
dataset['JGENE_allele1']= dataset['JGENE_allele'].str.split(' ').str[1]
dataset['JGENE_Gene']= dataset['JGENE_allele1'].str.split('*').str[0]

## Adding length of the nucleotide and amino-acid CDR3 sequences

In [19]:
dataset['CDR3_NT_LEN'] = dataset['CDR3_IMGT'].apply(len)
dataset['NAME'] = dataset['VFAM'] + '_' + dataset['JGENE_Gene'] + '_' + dataset['CDR3_NT_LEN'].astype(str)
dataset['NAME_GENE'] = dataset['VGENE_Gene'] + '_' + dataset['JGENE_Gene'] + '_' + dataset['CDR3_NT_LEN'].astype(str)
dataset = dataset.reset_index()

## Identify the clusters based on the Levenshtein distance for nucleotide sequences

#### Apply the function to cluster the receptors based on equal CDR3 length at 80% nucleotide level similarity.

In [23]:
nns = dataset.groupby(['NAME_GENE'],as_index=False)["CDR3_IMGT"].apply(lambda x : fuzz1((x),80))## Translates the above information in a dataframe. Name and clustered CDR3 sequencesare retained only. 

data_nt = group2df_list(nns)
data_nt =data_nt.rename(columns={0: "CDR3_IMGT"})
## Adds a column with index of each cluster
lst_col = 'CDR3_IMGT'
 ## Separates each cluster CDR3 back to same row dimesions
cluster_df_nt = pd.DataFrame({col:np.repeat(data_nt[col].values, data_nt[lst_col].str.len())
            for col in data_nt.columns.difference([lst_col])
            }).assign(**{lst_col:np.concatenate(data_nt[lst_col].values)})[data_nt.columns.tolist()]

#cluster_df['CDR3_AA_LEN'] = cluster_df['Hcdr3_aa'].apply(len)
cluster_df_nt =cluster_df_nt.rename(columns={"index": "Cluster_nt"})
cluster_df_nt = cluster_df_nt.rename_axis(None)
# Drop duplicated rows from clustered data to properly merge the dataframes
df_nt = cluster_df_nt.drop_duplicates()
df_nt =df_nt.drop_duplicates(subset='CDR3_IMGT')
#dataset['index_col'] = dataset.index
# Merging the cluster information to original dataset.
df_LD = pd.merge(dataset, df_nt, on = (['CDR3_IMGT']), how = 'left')
df_LD = df_LD.drop('Cluster', 1)

In [24]:
## Cluster_nt columns is added to the dataframe that indicates the clonotype information of the BCR sequences.

df_LD

,index,Unnamed: 0,SequenceID,VDOMAIN_Functionality,VGENE_allele,VREGION_score,VREGION_identity,VREGION_identity_nt,VREGION_INS_DEL_identity,VREGION_INS_DEL_identity_nt,...,FR4_IMGT_AA,VGENE_allele1,VGENE_Gene,VFAM,JGENE_allele1,JGENE_Gene,CDR3_NT_LEN,NAME,NAME_GENE,Cluster_nt
0,0,0,AAAGTAGAGACTAGAT-1_contig_1,productive,"Homsap IGKV1-39*01 F, or Homsap IGKV1D-39*01 F",1269.0,94.98,265/279 nt,NaN,NaN,...,FGPGTKVDIK,IGKV1-39*01,IGKV1-39,IGKV1,IGKJ3*01,IGKJ3,27,IGKV1_IGKJ3_27,IGKV1-39_IGKJ3_27,3766
1,1,1,AAAGTAGAGACTAGAT-1_contig_2,productive (see comment),Homsap IGHV4-61*01 F,1275.0,93.13,271/291 nt,92.78,270/291 nt,...,WGQGTTLIVSS,IGHV4-61*01,IGHV4-61,IGHV4,IGHJ6*02,IGHJ6,54,IGHV4_IGHJ6_54,IGHV4-61_IGHJ6_54,3292
2,2,3,AACCATGCAAGACGTG-1_contig_1,productive,Homsap IGHV4-61*01 F,1239.0,91.75,267/291 nt,NaN,NaN,...,WGLGTTVTVSA,IGHV4-61*01,IGHV4-61,IGHV4,IGHJ6*02,IGHJ6,48,IGHV4_IGHJ6_48,IGHV4-61_IGHJ6_48,3286
3,3,4,AACCATGCAAGACGTG-1_contig_2,productive,Homsap IGKV3-20*01 F,1257.0,93.97,265/282 nt,NaN,NaN,...,FGGGTKVEIK,IGKV3-20*01,IGKV3-20,IGKV3,IGKJ4*01,IGKJ4,30,IGKV3_IGKJ4_30,IGKV3-20_IGKJ4_30,4304
4,4,6,ACAGCTAAGGGATCTG-1_contig_1,productive,Homsap IGHV1-2*02 F,1188.0,90.28,260/288 nt,NaN,NaN,...,WGQGTLVSVSS,IGHV1-2*02,IGHV1-2,IGHV1,IGHJ4*02,IGHJ4,48,IGHV1_IGHJ4_48,IGHV1-2_IGHJ4_48,246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13752,13752,16692,TCATTACAGGTAAACT-1_contig_1,productive,Homsap IGLV3-1*01 F,1341.0,97.85,273/279 nt,NaN,NaN,...,FGTGTRVTVL,IGLV3-1*01,IGLV3-1,IGLV3,IGLJ1*01,IGLJ1,27,IGLV3_IGLJ1_27,IGLV3-1_IGLJ1_27,4843
13753,13753,16694,TGCGGGTAGGACACCA-1_contig_1,productive,Homsap IGKV1-9*01 F,1278.0,95.34,266/279 nt,NaN,NaN,...,FGPGTRVDFK,IGKV1-9*01,IGKV1-9,IGKV1,IGKJ3*01,IGKJ3,27,IGKV1_IGKJ3_27,IGKV1-9_IGKJ3_27,3922
13754,13754,16695,TGCGGGTAGGACACCA-1_contig_2,productive,Homsap IGHV2-5*01 F,1140.0,87.97,256/291 nt,NaN,NaN,...,WGQGILVIVSS,IGHV2-5*01,IGHV2-5,IGHV2,IGHJ4*02,IGHJ4,36,IGHV2_IGHJ4_36,IGHV2-5_IGHJ4_36,714
13755,13755,16698,TGGCTGGAGTTGCAGG-1_contig_1,productive,"Homsap IGHV3-23*01 F, or Homsap IGHV3-23D*01 F",1305.0,94.79,273/288 nt,NaN,NaN,...,WGQGTLVTVSS,IGHV3-23*01,IGHV3-23,IGHV3,IGHJ4*02,IGHJ4,48,IGHV3_IGHJ4_48,IGHV3-23_IGHJ4_48,1320
